In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
import itertools
from scipy import stats

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [22]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [23]:
def prepare_users_df(interactions_df):
    inter_df_copy = interactions_df.copy()
    inter_df_copy['val'] = 1
    
    term_df = inter_df_copy.loc[:, ['user_id', 'term', 'val']]
    term_df = term_df.pivot_table(index='user_id', columns='term', values='val', aggfunc='count')
    term_df = term_df / term_df.sum(axis=1).values.reshape(-1, 1)
    term_df = term_df.rename_axis(None, axis=1).fillna(0)    
    term_df = term_df.add_prefix('user_term_')
    
    length_of_stay_df = inter_df_copy.loc[:, ['user_id', 'length_of_stay_bucket', 'val']]
    length_of_stay_df = length_of_stay_df.pivot_table(index='user_id', columns='length_of_stay_bucket', values='val', aggfunc='count')
    length_of_stay_df = length_of_stay_df / length_of_stay_df.sum(axis=1).values.reshape(-1, 1)
    length_of_stay_df = length_of_stay_df.rename_axis(None, axis=1).fillna(0)
    length_of_stay_df = length_of_stay_df.add_prefix('user_length_of_stay_bucket_')

    rate_plan_df = inter_df_copy.loc[:, ['user_id', 'rate_plan', 'val']]
    rate_plan_df = rate_plan_df.pivot_table(index='user_id', columns='rate_plan', values='val', aggfunc='count')
    rate_plan_df = rate_plan_df / rate_plan_df.sum(axis=1).values.reshape(-1, 1)
    rate_plan_df = rate_plan_df.rename_axis(None, axis=1).fillna(0)    
    rate_plan_df = rate_plan_df.add_prefix('user_rate_plan_')
    
    room_segment_df = inter_df_copy.loc[:, ['user_id', 'room_segment', 'val']]
    room_segment_df = room_segment_df.pivot_table(index='user_id', columns='room_segment', values='val', aggfunc='count')
    room_segment_df = room_segment_df / room_segment_df.sum(axis=1).values.reshape(-1, 1)
    room_segment_df = room_segment_df.rename_axis(None, axis=1).fillna(0)
    room_segment_df = room_segment_df.add_prefix('user_room_segment_')

    n_people_bucket_df = inter_df_copy.loc[:, ['user_id', 'n_people_bucket', 'val']]
    n_people_bucket_df = n_people_bucket_df.pivot_table(index='user_id', columns='n_people_bucket', values='val', aggfunc='count')
    n_people_bucket_df = n_people_bucket_df / n_people_bucket_df.sum(axis=1).values.reshape(-1, 1)
    n_people_bucket_df = n_people_bucket_df.rename_axis(None, axis=1).fillna(0)
    n_people_bucket_df = n_people_bucket_df.add_prefix('user_n_people_bucket_')

    weekend_stay_df = inter_df_copy.loc[:, ['user_id', 'weekend_stay', 'val']]
    weekend_stay_df = weekend_stay_df.pivot_table(index='user_id', columns='weekend_stay', values='val', aggfunc='count')
    weekend_stay_df = weekend_stay_df / weekend_stay_df.sum(axis=1).values.reshape(-1, 1)
    weekend_stay_df = weekend_stay_df.rename_axis(None, axis=1).fillna(0)    
    weekend_stay_df = weekend_stay_df.add_prefix('user_weekend_stay_')
    
    users_df = pd.merge(term_df, length_of_stay_df, on='user_id')
    users_df = users_df.merge(rate_plan_df, on='user_id')
    users_df = users_df.merge(room_segment_df, on='user_id')
    users_df = users_df.merge(n_people_bucket_df, on='user_id')
    users_df = users_df.merge(weekend_stay_df, on='user_id')
    users_df = users_df.reset_index()

    # add addictional feature for Carthesian product
    #     users_df['popularity'] = '1'
    
    user_features = users_df.columns.tolist()
    user_features.remove('user_id')
    
    return users_df.astype("float"), user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_WinterVacation', 'user_term_Easter', 'user_term_OffSeason', 'user_term_HighSeason', 'user_term_LowSeason', 'user_term_MayLongWeekend', 'user_term_NewYear', 'user_term_Christmas', 'user_length_of_stay_bucket_[0-1]', 'user_length_of_stay_bucket_[2-3]', 'user_length_of_stay_bucket_[4-7]', 'user_length_of_stay_bucket_[8-inf]', 'user_rate_plan_Standard', 'user_rate_plan_Nonref', 'user_room_segment_[0-160]', 'user_room_segment_[160-260]', 'user_room_segment_[260-360]', 'user_room_segment_[360-500]', 'user_room_segment_[500-900]', 'user_n_people_bucket_[1-1]', 'user_n_people_bucket_[2-2]', 'user_n_people_bucket_[3-4]', 'user_n_people_bucket_[5-inf]', 'user_weekend_stay_True', 'user_weekend_stay_False']


,user_id,user_term_WinterVacation,user_term_Easter,user_term_OffSeason,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_Christmas,user_length_of_stay_bucket_[0-1],...,user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_room_segment_[500-900],user_n_people_bucket_[1-1],user_n_people_bucket_[2-2],user_n_people_bucket_[3-4],user_n_people_bucket_[5-inf],user_weekend_stay_True,user_weekend_stay_False
0,1.0,0.090909,0.0,0.681818,0.090909,0.136364,0.000000,0.000000,0.0,0.000000,...,0.863636,0.136364,0.0,0.0,0.000000,0.727273,0.181818,0.090909,0.772727,0.227273
40,50.0,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.0,0.000000,...,0.565217,0.434783,0.0,0.0,0.000000,0.173913,0.521739,0.304348,0.782609,0.217391
84,96.0,0.090909,0.0,0.681818,0.136364,0.045455,0.045455,0.000000,0.0,0.272727,...,0.863636,0.090909,0.0,0.0,0.045455,0.272727,0.590909,0.090909,0.727273,0.272727
102,115.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.500000,0.500000,0.0,0.0,0.500000,0.000000,0.000000,0.500000,0.000000,1.000000
371,706.0,0.095238,0.0,0.511905,0.190476,0.142857,0.047619,0.011905,0.0,0.297619,...,0.857143,0.107143,0.0,0.0,0.120482,0.156627,0.590361,0.132530,0.583333,0.416667
1383,1736.0,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.0,0.241379,...,0.931034,0.068966,0.0,0.0,0.379310,0.413793,0.206897,0.000000,0.448276,0.551724
7301,7779.0,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,...,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.750000,0.250000,0.750000,0.250000


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [24]:
def prepare_items_df(interactions_df):
    missing_elements = np.array([])
    
    inter_df_copy = interactions_df.copy()
    inter_df_copy['val'] = 1

    items_df_item_id = pd.DataFrame(data=interactions_df['item_id'].unique(), columns=['item_id'])
    # , aggfunc=lambda x: list(stats.mode(x.values).mode)

    missing_elements = list(set(inter_df_copy['term'].unique().categories.tolist())-set(inter_df_copy['term'].unique()))
    term_df = inter_df_copy.pivot_table(index='item_id', columns='term', values='val')
    term_df = term_df / term_df.sum(axis=1).values.reshape(-1, 1)
    term_df = term_df.rename_axis(None, axis=1).fillna(0)
    term_df[missing_elements] = 0
    term_df = term_df.add_prefix('item_term_')

    missing_elements = list(set(inter_df_copy['length_of_stay_bucket'].unique().categories.tolist())-set(inter_df_copy['length_of_stay_bucket'].unique()))
    length_of_stay_df = inter_df_copy.pivot_table(index='item_id', columns='length_of_stay_bucket', values='val')
    length_of_stay_df = length_of_stay_df / length_of_stay_df.sum(axis=1).values.reshape(-1, 1)
    length_of_stay_df = length_of_stay_df.rename_axis(None, axis=1).fillna(0)
    term_df[missing_elements] = 0
    length_of_stay_df = length_of_stay_df.add_prefix('item_length_of_stay_')

    missing_elements = list(set(inter_df_copy['rate_plan'].unique().categories.tolist())-set(inter_df_copy['rate_plan'].unique()))
    rate_plan_df = inter_df_copy.pivot_table(index='item_id', columns='rate_plan', values='val')
    rate_plan_df = rate_plan_df / rate_plan_df.sum(axis=1).values.reshape(-1, 1)
    rate_plan_df = rate_plan_df.rename_axis(None, axis=1).fillna(0)
    term_df[missing_elements] = 0
    rate_plan_df = rate_plan_df.add_prefix('item_rate_plan_')

    missing_elements = list(set(inter_df_copy['room_segment'].unique().categories.tolist())-set(inter_df_copy['room_segment'].unique()))
    room_segment_df = inter_df_copy.pivot_table(index='item_id', columns='room_segment', values='val')
    room_segment_df = room_segment_df / room_segment_df.sum(axis=1).values.reshape(-1, 1)
    room_segment_df = room_segment_df.rename_axis(None, axis=1).fillna(0)    
    room_segment_df[missing_elements] = 0
    room_segment_df = room_segment_df.add_prefix('item_room_segment_')
    
    missing_elements = list(set(inter_df_copy['n_people_bucket'].unique().categories.tolist())-set(inter_df_copy['n_people_bucket'].unique()))
    n_people_df = inter_df_copy.pivot_table(index='item_id', columns='n_people_bucket', values='val')
    n_people_df = n_people_df / n_people_df.sum(axis=1).values.reshape(-1, 1)
    n_people_df = n_people_df.rename_axis(None, axis=1).fillna(0)
    room_segment_df[missing_elements] = 0
    n_people_df = n_people_df.add_prefix('item_n_people_')

    missing_elements = list(set(inter_df_copy['n_people_bucket'].unique().categories.tolist())-set(inter_df_copy['n_people_bucket'].unique()))
    weekend_stay_df = inter_df_copy.pivot_table(index='item_id', columns='weekend_stay', values='val')
    weekend_stay_df = weekend_stay_df / weekend_stay_df.sum(axis=1).values.reshape(-1, 1)
    weekend_stay_df = weekend_stay_df.rename_axis(None, axis=1).fillna(0)
    room_segment_df[missing_elements] = 0
    weekend_stay_df = weekend_stay_df.add_prefix('item_weekend_stay_')

    #items_df = pd.concat([lenght_of_stay, room_segment, n_people], axis=1)
    

    items_df = pd.merge(term_df, length_of_stay_df, on='item_id')
    items_df = items_df.merge(rate_plan_df, on='item_id')
    items_df = items_df.merge(room_segment_df, on='item_id')
    items_df = items_df.merge(n_people_df, on='item_id')
    items_df = items_df.merge(weekend_stay_df, on='item_id')
    items_df = items_df.reset_index()

    # item popularity
    #     item_popularity = inter_df_copy.loc[:, ['item_id', 'user_id']].groupby(by='item_id', as_index=True).count()
    #     item_popularity = item_popularity.rename(columns={'user_id': 'item_popularity'})

    #     items_df = pd.merge(items_df, item_popularity, on='item_id', how='left')

    item_features = items_df.columns.tolist()
    item_features.remove('item_id')

    return items_df.astype("float"), item_features

items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['item_term_WinterVacation', 'item_term_Easter', 'item_term_OffSeason', 'item_term_HighSeason', 'item_term_LowSeason', 'item_term_MayLongWeekend', 'item_term_NewYear', 'item_term_Christmas', 'item_length_of_stay_[0-1]', 'item_length_of_stay_[2-3]', 'item_length_of_stay_[4-7]', 'item_length_of_stay_[8-inf]', 'item_rate_plan_Standard', 'item_rate_plan_Nonref', 'item_room_segment_[0-160]', 'item_room_segment_[160-260]', 'item_room_segment_[260-360]', 'item_room_segment_[360-500]', 'item_room_segment_[500-900]', 'item_n_people_[1-1]', 'item_n_people_[2-2]', 'item_n_people_[3-4]', 'item_n_people_[5-inf]', 'item_weekend_stay_True', 'item_weekend_stay_False']


,item_id,item_term_WinterVacation,item_term_Easter,item_term_OffSeason,item_term_HighSeason,item_term_LowSeason,item_term_MayLongWeekend,item_term_NewYear,item_term_Christmas,item_length_of_stay_[0-1],...,item_room_segment_[160-260],item_room_segment_[260-360],item_room_segment_[360-500],item_room_segment_[500-900],item_n_people_[1-1],item_n_people_[2-2],item_n_people_[3-4],item_n_people_[5-inf],item_weekend_stay_True,item_weekend_stay_False
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        user_id = interactions_df['user_id'].unique().tolist()
        item_id = interactions_df['item_id'].unique().tolist()

        touple_of_all_combinations = list(itertools.product(user_id, item_id))  # list of all combinations
        touple_of_interactions_pair = [tuple(x) for x in interactions_df.loc[:, ['user_id', 'item_id']].to_numpy()]  # list of interactions combinations 
        not_appear_combinations = list(set(touple_of_all_combinations) - set(touple_of_interactions_pair))  # defference of sets get combinations not appear in interactions_df

        rng = np.random.RandomState(seed=self.seed)
        random_ids_combinations = rng.choice(list(range(len(not_appear_combinations))), size=int(self.n_neg_per_pos) * len(interactions_df), replace=False) # generate set of negative interactions
        negative_interactions = [not_appear_combinations[index] for index in random_ids_combinations]  # map index to its value
        negative_interactions = [list(tuple_combination)+[0] for tuple_combination in negative_interactions]  # change strusture of touple, add 0 as sufix

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])

        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.
        
        # prepare average features
        recommend_df = items_df.copy()
        
        users_df = pd.merge(users_df, self.users_df, on='user_id', how='left')
        users_df = users_df.fillna(self.users_df.mean())
        
        items_df, item_features = prepare_items_df(items_df)
        
        # Score the items
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            carthesian_product = pd.merge(items_df, user.to_frame().transpose(), how='cross')
            carthesian_product = carthesian_product.drop(['user_id', 'item_id'], axis=1)

            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            scores = self.model.predict(carthesian_product.loc[:, self.user_features + item_features].values)

            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [26]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [7]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [8]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,54,0.997021,OffSeason,[0-1],Standard,[160-260],[2-2],True
1,1.0,50,0.993057,HighSeason,[4-7],Standard,[160-260],[5-inf],False
2,1.0,60,0.985957,WinterVacation,[2-3],Nonref,[160-260],[2-2],True
3,1.0,22,0.979874,OffSeason,[2-3],Standard,[160-260],[3-4],True
4,1.0,75,0.978921,OffSeason,[4-7],Nonref,[0-160],[2-2],True
5,1.0,23,0.963803,OffSeason,[4-7],Standard,[160-260],[2-2],True
6,1.0,24,0.960419,OffSeason,[2-3],Standard,[260-360],[3-4],True
7,1.0,14,0.957904,OffSeason,[4-7],Standard,[160-260],[3-4],True
8,1.0,6,0.946597,OffSeason,[2-3],Standard,[260-360],[5-inf],True
9,1.0,71,0.946375,OffSeason,[2-3],Standard,[260-360],[5-inf],False


# Tuning method

In [27]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [28]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [371]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|█████████████████████████████████████████████| 10/10 [46:10<00:00, 277.08s/trial, best loss: -0.12145577655522641]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.0,0.039375,0.076035,0.114053,0.0,0.024576,0.040201,0.052227


Best parameters:
{'n_neg_per_pos': 10.0}


In [29]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|                                                                          | 0/100 [00:03<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "C:\Users\Windows 10\AppData\Local\Temp\ipykernel_9220\3138901096.py", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 300, in run
    self.serial_evaluate()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 178, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 892, in evaluate
    r

In [18]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|█████████████████████████████████████████| 100/100 [9:24:26<00:00, 338.67s/trial, best loss: -0.12484130342784595]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.001697,0.068907,0.078751,0.125594,0.001697,0.039969,0.043926,0.058691


Best parameters:
{'max_depth': 7.0, 'min_samples_split': 19.0, 'n_estimators': 300.0, 'n_neg_per_pos': 3.0}


In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [16]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 10})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.0,0.039375,0.076035,0.114053,0.0,0.024576,0.040201,0.052227


In [37]:
cb_user_item_recommender = RandomForestCBUIRecommender(
    **{'max_depth': 7.0, 'min_samples_split': 19.0, 'n_estimators': 300.0, 'n_neg_per_pos': 3.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
randomForest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

randomForest_cbui_tts_results = pd.DataFrame(
    randomForest_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(randomForest_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.002716,0.070604,0.083164,0.125933,0.002716,0.041326,0.046304,0.059609


In [34]:
cb_user_item_recommender = XGBoostCBUIRecommender(
    **{'learning_rate': 0.02, 'max_depth': 7.0, 'min_samples_split': 18.0, 'n_estimators': 130.0, 'n_neg_per_pos': 4.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
xgboost_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

xgboost_cbui_tts_results = pd.DataFrame(
    xgboost_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(xgboost_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.038018,0.052274,0.087576,0.126952,0.038018,0.045724,0.059604,0.071862


In [12]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468


In [38]:
tts_results = pd.concat([linear_cbui_tts_results, randomForest_cbui_tts_results, xgboost_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.000000,0.039375,0.076035,0.114053,0.000000,0.024576,0.040201,0.052227
1,RandomForestCBUIRecommender,0.002716,0.070604,0.083164,0.125933,0.002716,0.041326,0.046304,0.059609
2,XGBoostCBUIRecommender,0.038018,0.052274,0.087576,0.126952,0.038018,0.045724,0.059604,0.071862
3,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468
